In [1]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Reload environment
exit()

     |████████████████████████████████| 735.5 MB 13 kB/s 
     |████████████████████████████████| 7.1 MB 27.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


Import Libraries

In [1]:
import numpy
import torch
import torch.nn as nn
import pdb
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import time
from torch.autograd import Variable

In [2]:
import pandas as pd
import numpy as np

In [5]:
import sys
import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets

Parameters of LSTM

In [13]:
batch_size = 512
embedding_dim = 300
dropout_ratio = 0.2
hidden_dim = 256
epochs = 10
lr = 0.001
bidirect = True
opt_name = 'Adam'

Setting device to CPU

In [7]:
device = torch.device('cpu')

Parameters of TFIDF_Logistic_Reg

In [8]:
max_iter = 300
l1_ratio = None
tol = 0.001

Importing the classes of the dataset and the network

In [9]:
class SNLI():
    def __init__(self, batch_size, device):
        self.inputs = data.Field(lower=True, tokenize = None, batch_first = True)
        self.labels = data.Field(sequential = False, unk_token = None, is_target = True)
        self.train, self.val, self.test = datasets.SNLI.splits(self.inputs, self.labels)
        self.inputs.build_vocab(self.train, self.val)
        self.labels.build_vocab(self.train)
        self.train_iter, self.val_iter, self.test_iter = data.Iterator.splits((self.train, self.val, self.test), batch_size = batch_size, device=device)
    
    def vocabulary_size(self):
        return len(self.inputs.vocab)
    
    def out_dim(self):
        return len(self.labels.vocab)
    
    def labels(self):
        return self.labels.vocab.stoi

In [10]:
class BiLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, dropout_ratio, hidden_dim, out_dim, bidirect):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.projection = nn.Linear(embedding_dim, 300)
        self.dropout = nn.Dropout(p = dropout_ratio)
        self.lstm = nn.LSTM(300, hidden_dim, 3, bidirectional=bidirect)
        self.relu = nn.ReLU()
        self.out = nn.Sequential(
            nn.Linear(2*512, 512),
			self.relu,
			self.dropout,
			nn.Linear(512, 256),
			self.relu,
			self.dropout,
			nn.Linear(256, 256),
			self.relu,
			self.dropout,
			nn.Linear(256, out_dim)
        )
        pass

    def forward(self, batch):
        premise_embedding = self.embedding(batch.premise)
        hypothesis_embedding = self.embedding(batch.hypothesis)

        premise_projection = self.relu(self.projection(premise_embedding))
        hypothesis_projection = self.relu(self.projection(hypothesis_embedding))

        encoded_premise, _ = self.lstm(premise_projection)
        encoded_hypothesis, _ = self.lstm(hypothesis_projection)

        premise = encoded_premise.sum(dim=1)
        hypothesis = encoded_hypothesis.sum(dim=1)

        combined = torch.cat((premise, hypothesis), 1)

        return self.out(combined)

In [11]:
def test(model, dataset):
    model.eval()
    dataset.test_iter.init_epoch()
    y_gt_label = []
    y_pred_label = []
    correct = 0
    total = 0
    n_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataset.test_iter):
            prediction = model(batch)
            yt_pred = Variable(prediction)
            yt_pred = F.softmax(yt_pred, dim=1)
            yt_pred_label = torch.argmax(yt_pred, dim=1)
            yt_gt = Variable(batch.label)
            loss = F.cross_entropy(prediction, batch.label)
            correct += (torch.max(prediction, 1)[1].view(batch.label.size()) == batch.label).sum().item()
            total += batch.batch_size
            n_loss += loss.item()

            # Add the labels
            y_gt_label += list(yt_gt.numpy())
            y_pred_label += list(yt_pred_label.numpy())
    
    test_loss = n_loss/total;
    test_acc = (correct/total) * 100.
    return test_loss, test_acc, y_gt_label, y_pred_label

Getting the test data from pytorch

In [15]:
dataset = SNLI(batch_size, device)
out_dim = dataset.out_dim()
vocab_size = dataset.vocabulary_size()

In [20]:
if __name__ == "__main__":

    from google.colab import drive
    drive.mount('/content/drive')
    
    # Testing the LSTM model

    # Loading the model using the parameters needed
    filename = "/content/drive/MyDrive/" + '{0}_{1}_{2}_{3}_{4}_{5}_{6}_bidirect.pt'.format(batch_size, embedding_dim, dropout_ratio, hidden_dim, epochs, opt_name, lr)
    model = BiLSTM(vocab_size, embedding_dim, dropout_ratio, hidden_dim, out_dim, bidirect)
    model.to(device)
    model.load_state_dict(torch.load(filename, map_location=torch.device('cpu')))
    test_loss, test_accuracy, gt, pred = test(model, dataset)
    # print("Test loss = {}, Test accuracy = {}".format(test_loss, test_accuracy))

    # Writing the output from LSTM onto a text file
    labels = ['entailment', 'contradiction', 'neutral']
    with open("/content/drive/MyDrive/LSTM.txt", 'w') as f:
        f.write("Loss on Test Data : {}\n".format(test_loss))
        f.write("Accuracy on Test Data : {}\n".format(test_accuracy))
        f.write("gt_label,pred_label \n")
        for idx in range(len(gt)):
            f.write("{},{}\n".format(labels[gt[idx]], labels[pred[idx]]))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
